In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from shared.pipelines import get_keras_pipeline
from shared.data import load_imdb

Using TensorFlow backend.


In [3]:
import en_vectors_web_lg
nlp = en_vectors_web_lg.load()

In [4]:
MAX_WORDS_IN_SENTENCE = 200
LIMIT = -1
model_path = '../dist/models/imdb_{}.h5'.format(LIMIT if LIMIT > 0 else 'all')

In [5]:
import matplotlib.pyplot as plt
from keras import models


def plot_training(history):
    plt.plot(history.history['loss'], c='red')
    

def train_model(limit, max_words_in_sentence, **train_params): 
    train_params['validation_split'] = 0
    X, y = load_imdb('../data/aclImdb/train', limit)
    pipeline = get_keras_pipeline(nlp, max_words_in_sentence, **train_params).fit(X, y)
    model_step = pipeline.steps[-1][1]
    plot_training(model_step.history)
    return model_step.model


def load_or_train_model(model_path, limit, max_words_in_sentence, **train_params):
    try:
        print("Loading cached model...")
        return models.load_model(model_path)
    except:
        print("Unable to find cached model, training...")
        model = train_model(limit, max_words_in_sentence, **train_params)
        models.save_model(model, model_path)
        return model

In [11]:
model = load_or_train_model(
    model_path, limit=LIMIT, epochs=50, 
    max_words_in_sentence=MAX_WORDS_IN_SENTENCE)
pipeline = get_keras_pipeline(nlp, model=model, max_words_in_sentence=MAX_WORDS_IN_SENTENCE)

Loading cached model...


In [7]:
X, y = load_imdb('../data/aclImdb/test', limit=1000)
pipeline.score(X, y)

1000/1000 [==============================] - 0s 165us/step


0.80000000143051142

In [10]:
import numpy as np

pipeline.predict_proba(np.array([
    "That movie is realy good",
    "I don't like it at all",
], dtype='object'))

array([[ 0.38960242,  0.61039758],
       [ 0.38748628,  0.61251372]], dtype=float32)